# Billboard Functions

In [3]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
import urllib
from collections import Counter

################################################################################
## General Stuff
################################################################################
from searchUtils import findNearest
from searchUtils import findExt
from searchUtils import findPatternExt
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from time import sleep

################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb



from multiArtist import multiartist
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-08-27 21:01:45.094853


# Global Vars

# Starter Class Section

In [4]:
basedir = "/Volumes/Piggy/Charts/"
try:
    filename = glob(join(basedir, "data", "billboard", "starter.html"))[0]
except:
    print("Could not find starter HTML file!")
fdata = getHTML(filename)

#### Extra

In [4]:
chartData  = {}
dirname = None
baseURL = "https://www.billboard.com"
for iul,ul in enumerate(fdata.findAll("ul")):
    lis = ul.findAll("li", {"class": "chart-group__item"})
    for j,li in enumerate(lis):  
        a = li.find('a')
        if a is not None:
            href = a['href']
            text = a.text
            subdir = getDirname(href)[1:]
            chartData[text] = [baseURL, subdir, getBasename(href)]

In [9]:
#chartData

# Download Yearly Information

In [ ]:
from time import sleep
years = [str(x) for x in range(1958,2014)]
years = ['2019']
for year in years:
    url="https://www.billboard.com/archive/charts/{0}".format(year)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    savedir = join(basedir, "data", "billboard", "yearly")
    savename = join(savedir, "{0}.p".format(year))
    if isFile(savename):
        continue
    
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(1)

# Parse Yearly Information

In [10]:
downloads = {}
baseURL = "https://www.billboard.com"
for ifile in sorted(glob(join(basedir, "data", "billboard", "yearly", "*.p"))):
    year = getBaseFilename(ifile)
    if downloads.get(year) is None:
        downloads[year] = {}
    fdata = getHTML(ifile)
    for iul,ul in enumerate(fdata.findAll("ul")):
        lis = ul.findAll("li", {"class": "chart-group__item"})
        for j,li in enumerate(lis):  
            a = li.find('a')
            if a is not None:
                href = a['href']
                text = a.text
                subdir = getDirname(href)[1:]
                downloads[year][href] = [baseURL, subdir, getBasename(href)]

In [11]:
downloads.keys()

dict_keys(['1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'])

# Download Category Data

In [ ]:
baseURL = "https://www.billboard.com"

for year, yearData in downloads.items():   
    for href, hrefData in yearData.items():
        url  = "{0}/{1}".format(baseURL, href)
        year = getBasename(hrefData[1])
        category = hrefData[2]

        savedir  = join(basedir, "data", "billboard", "categories")
        savename = join(savedir, "{0}-{1}.p".format(year, category))

        if isFile(savename):
            continue

        print("  Trying to download and save {0}".format(savename))
        try:

            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
            headers={'User-Agent':user_agent,} 

            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need
        except:
            continue

        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(1)        

# Parse Category Data

In [6]:
data = {}

baseURL = "https://www.billboard.com"
names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
categories = set([x[5:] for x in names])

albumCategories  = [x for x in categories if x.endswith("albums")]
songCategories   = [x for x in categories if not x.endswith("albums")]


names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
years = sorted(set([x[:4] for x in names]))

print("There are {0} years".format(len(years)))
print("There are {0} charts".format(len(categories)))
print("There are {0} album charts".format(len(albumCategories)))
print("There are {0} song charts".format(len(songCategories)))

There are 62 years
There are 178 charts
There are 32 album charts
There are 146 song charts


In [13]:
#categories

In [7]:
def parseBillboardFile(ifile):
    fdata = getHTML(ifile)
    data  = {}
    for i,table in enumerate(fdata.findAll('table')):
        ths = table.findAll("th")
        trs = table.findAll("tr")

        headers = [x.text for x in ths]
        for j,tr in enumerate(trs[1:]):
            tds  = tr.findAll('td')
            vals = [x.text for x in tds]
            #print(headers,vals)

            if len(vals) == 3:
                date,name,artist = vals
                try:
                    date = getDateTime(", ".join([date, year]))
                except:
                    print(vals)
                    print(", ".join([date, year]))

                    1/0
            elif len(vals) == 1:
                date = vals[0]
                try:
                    date = getDateTime(", ".join([date, year]))
                except:
                    print(vals)
                    1/0

            if not isDate(date):
                raise ValueError("Could not form date for {0}".format(date))

            date = printDateTime(date)
            if data.get(date) is not None:
                raise ValueError("Already seen this date!!!")
            data[date] = {"Artist": artist, "Name": name}
            
            #print("{0: <12}{1}".format(date, data[date]))
            continue
            
            
            if data.get(date) is None:
                data[date] = {}
            #print("{0: <20}{1: <20}{2: <20}{3: <20}".format(artist, name, category, date))
            #continue
            if data.get(artist) is None:
                data[artist] = {}
                #print("Artist: {0}".format(artist))
            if data[artist].get(name) is None:
                data[artist][name] = {}
                #print("\tSong: {0} ({1})".format(name, len(data[artist])))
            if data[artist][name].get(category) is None:
                data[artist][name][category] = []
            data[artist][name][category].append([date])

    return data

In [19]:
catdir = join(basedir, "data", "billboard", "categories")
for year in years:
    data  = {}
    files = findPatternExt(catdir, pattern=year, ext='.p')    
    for ifile in files:
        chart   = getBaseFilename(ifile)
        chart   = "-".join(chart.split('-')[1:])
        results = parseBillboardFile(ifile)
        data[chart] = results
        print("{0: <10}{1: <30}{2}".format(year, chart, len(results)))
        
    savedir = join(basedir, "data", "billboard", "results")
    savename = join(savedir, "{0}.p".format(year))
    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)

1958      country-songs                 11
1958      hot-100                       22
1958      r-b-hip-hop-songs             11
Saving /Volumes/Piggy/Charts/data/billboard/results/1958.p
  --> This file is 928B.
1959      country-songs                 52
1959      hot-100                       52
1959      r-b-hip-hop-songs             52
Saving /Volumes/Piggy/Charts/data/billboard/results/1959.p
  --> This file is 2.8kB.
1960      country-songs                 52
1960      hot-100                       52
1960      r-b-hip-hop-songs             52
Saving /Volumes/Piggy/Charts/data/billboard/results/1960.p
  --> This file is 2.8kB.
1961      adult-contemporary            24
1961      country-songs                 51
1961      hot-100                       52
1961      r-b-hip-hop-songs             51
Saving /Volumes/Piggy/Charts/data/billboard/results/1961.p
  --> This file is 3.3kB.
1962      adult-contemporary            52
1962      country-songs                 52
1962      hot-10

1981      dance-club-play-songs         52
1981      hot-100                       52
1981      hot-mainstream-rock-tracks    41
1981      r-b-hip-hop-albums            52
1981      r-b-hip-hop-songs             52
Saving /Volumes/Piggy/Charts/data/billboard/results/1981.p
  --> This file is 9.3kB.
1982      adult-contemporary            52
1982      billboard-200                 52
1982      country-albums                52
1982      country-songs                 52
1982      dance-club-play-songs         52
1982      hot-100                       52
1982      hot-mainstream-rock-tracks    52
1982      r-b-hip-hop-albums            52
1982      r-b-hip-hop-songs             52
Saving /Volumes/Piggy/Charts/data/billboard/results/1982.p
  --> This file is 9.3kB.
1983      adult-contemporary            51
1983      billboard-200                 51
1983      christian-albums              2
1983      country-albums                51
1983      country-songs                 51
1983      danc

1992      country-songs                 52
1992      dance-club-play-songs         52
1992      gospel-albums                 26
1992      heatseekers-albums            52
1992      holiday-albums                6
1992      hot-100                       52
1992      hot-mainstream-rock-tracks    52
1992      hot-r-and-b-hip-hop-airplay   39
1992      latin-pop-albums              26
1992      latin-songs                   52
1992      new-age-albums                26
1992      pop-songs                     13
1992      r-b-hip-hop-albums            52
1992      r-b-hip-hop-songs             52
1992      radio-songs                   52
1992      rap-song                      52
1992      regional-mexican-albums       26
1992      rhythmic-40                   13
1992      top-album-sales               52
1992      tropical-albums               26
1992      world-albums                  26
Saving /Volumes/Piggy/Charts/data/billboard/results/1992.p
  --> This file is 20.7kB.
1993      ad

1997      heatseekers-albums            52
1997      holiday-albums                8
1997      hot-100                       52
1997      hot-mainstream-rock-tracks    52
1997      hot-r-and-b-hip-hop-airplay   52
1997      jazz-albums                   52
1997      kids-albums                   52
1997      latin-airplay                 52
1997      latin-albums                  52
1997      latin-pop-albums              52
1997      latin-pop-songs               52
1997      latin-songs                   52
1997      new-age-albums                52
1997      pop-songs                     52
1997      r-b-hip-hop-albums            52
1997      r-b-hip-hop-songs             52
1997      radio-songs                   52
1997      rap-song                      52
1997      reggae-albums                 52
1997      regional-mexican-albums       52
1997      regional-mexican-songs        52
1997      rhythmic-40                   52
1997      top-album-sales               52
1997      tr

2001      rhythmic-40                   52
2001      soundtracks                   27
2001      top-album-sales               52
2001      tropical-albums               52
2001      tropical-songs                52
2001      world-albums                  52
Saving /Volumes/Piggy/Charts/data/billboard/results/2001.p
  --> This file is 35.9kB.
2002      adult-contemporary            52
2002      adult-pop-songs               52
2002      alternative-songs             52
2002      billboard-200                 52
2002      bluegrass-albums              24
2002      blues-albums                  52
2002      canadian-albums               52
2002      catalog-albums                52
2002      christian-albums              52
2002      classical-albums              52
2002      country-airplay               52
2002      country-albums                52
2002      country-songs                 52
2002      dance-club-play-songs         52
2002      dance-electronic-albums       52
2002      g

...

2008      tastemaker-albums             52
2008      top-album-sales               52
2008      tropical-albums               52
2008      tropical-songs                52
2008      world-albums                  52
Saving /Volumes/Piggy/Charts/data/billboard/results/2008.p
  --> This file is 53.3kB.
2009      adult-contemporary            52
2009      adult-pop-songs               52
2009      alternative-albums            52
2009      alternative-songs             52
2009      americana-folk-albums         4
2009      billboard-200                 52
2009      bluegrass-albums              52
2009      blues-albums                  52
2009      canadian-albums               52
2009      canadian-hot-100              52
2009      catalog-albums                52
2009      christian-airplay             52
2009      christian-albums              52
2009      christian-songs               52
2009      classical-albums              52
2009      comedy-albums                 52
2009      co

2011      new-age-albums                51
2011      pop-songs                     51
2011      r-and-b-hip-hop-digital-song-sales51
2011      r-b-hip-hop-albums            51
2011      r-b-hip-hop-songs             51
2011      radio-songs                   51
2011      rap-albums                    51
2011      rap-song                      51
2011      reggae-albums                 51
2011      regional-mexican-albums       51
2011      regional-mexican-songs        51
2011      rhythmic-40                   51
2011      rock-airplay                  51
2011      rock-albums                   51
2011      rock-digital-song-sales       51
2011      rock-songs                    51
2011      social-50                     51
2011      soundtracks                   51
2011      tastemaker-albums             51
2011      top-album-sales               51
2011      tropical-albums               51
2011      tropical-songs                51
2011      vinyl-albums                  49
2011   

2014      americana-folk-albums         52
2014      artist-100                    24
2014      billboard-200                 52
2014      bluegrass-albums              52
2014      blues-albums                  52
2014      canadian-albums               52
2014      canadian-hot-100              52
2014      catalog-albums                52
2014      christian-airplay             52
2014      christian-albums              52
2014      christian-digital-song-sales  52
2014      christian-songs               52
2014      christian-streaming-songs     52
2014      classical-albums              52
2014      comedy-albums                 52
2014      country-airplay               52
2014      country-albums                52
2014      country-digital-song-sales    52
2014      country-songs                 52
2014      country-streaming-songs       52
2014      dance-club-play-songs         52
2014      dance-electronic-albums       52
2014      dance-electronic-digital-song-sales52
2014  

2016      christian-digital-song-sales  52
2016      christian-songs               52
2016      christian-streaming-songs     52
2016      classical-albums              52
2016      comedy-albums                 52
2016      country-airplay               52
2016      country-albums                52
2016      country-digital-song-sales    52
2016      country-songs                 52
2016      country-streaming-songs       52
2016      dance-club-play-songs         52
2016      dance-electronic-albums       52
2016      dance-electronic-digital-song-sales52
2016      dance-electronic-songs        52
2016      dance-electronic-streaming-songs52
2016      digital-albums                52
2016      digital-song-sales            52
2016      gospel-airplay                52
2016      gospel-albums                 52
2016      gospel-digital-song-sales     52
2016      gospel-songs                  52
2016      gospel-streaming-songs        52
2016      hard-rock-albums              52
2016

2018      country-streaming-songs       53
2018      dance-club-play-songs         53
2018      dance-electronic-albums       53
2018      dance-electronic-digital-song-sales53
2018      dance-electronic-songs        53
2018      dance-electronic-streaming-songs53
2018      digital-albums                53
2018      digital-song-sales            53
2018      gospel-airplay                53
2018      gospel-albums                 53
2018      gospel-digital-song-sales     53
2018      gospel-songs                  53
2018      gospel-streaming-songs        53
2018      hard-rock-albums              53
2018      heatseekers-albums            53
2018      holiday-albums                14
2018      holiday-season-digital-song-sales13
2018      holiday-songs                 5
2018      holiday-streaming-songs       5
2018      hot-100                       53
2018      hot-mainstream-rock-tracks    53
2018      hot-r-and-b-hip-hop-airplay   53
2018      independent-albums            53
201

# Aggregrate Charts

In [17]:
results = {}
print(basedir)
savedir = join(basedir, "data", "billboard", "results")
files = sorted(findExt(savedir, ".p"))

/Volumes/Piggy/Charts/


In [18]:
def getArtistRenames():
    artistRenames = getFile("masterRename.yaml")
    return artistRenames

In [44]:
from collections import Counter
def findArtistData(minYear=2019, maxYear=3000, country=None):
    artistRenames = getArtistRenames()
    renameStats   = Counter()
    fullChartData = {}
    chartData     = {}
    #country       = "USA"
    country       = country
    chartCounter  = Counter()
    nonUS = ["BLU", "DSI", "CRI", "HTA", "HTL", "LDT", "LSS", "COA", "JPN", "JSI"]
    nonUS += ["regional-mexican-albums", "latin-pop-albums", "latin-songs", "world-albums", "tropical-albums", "latin-albums"]
    nonUS += ['canadian-albums', 'canadian-hot-100', 'latin-digital-song-sales', 'japan-hot-100', 'latin-streaming-songs', 'china-v-chart', 'billboard-argentina-hot-100']
    nonUS += ['latin-airplay', 'latin-pop-songs', 'reggae-albums', 'regional-mexican-songs', "tropical-songs"]
    
    nonMusic = ["classical-albums"]

    mixMusic = ["bluegrass-albums", "gospel-airplay", "new-age-albums", "heatseekers-albums"]

    for ifile in files:
        print(ifile,end='\t')
        fdata = getFile(ifile)
        for chartName, cnameResults in fdata.items():
            for date, dResults in cnameResults.items():
                if minYear is not None:
                    if getDateTime(date).year < int(minYear):
                        continue
                if maxYear is not None:
                    if getDateTime(date).year > int(maxYear):
                        continue

                if chartName in nonMusic:
                    continue        
                if chartName in mixMusic:
                    continue
                if chartName.startswith("gospel"):
                    continue
                if chartName.startswith("bluegrass"):
                    continue
                if chartName.startswith("classical"):
                    continue
                if chartName.startswith("jazz"):
                    continue
                if chartName.startswith("holiday"):
                    continue
                if chartName.startswith("comedy"):
                    continue
                if chartName.startswith("soundtracks"):
                    continue
                if chartName.startswith("new-age"):
                    continue
                if chartName.startswith("heatseekers"):
                    continue
                if chartName.startswith("dance"):
                    continue
                if chartName.startswith("christian"):
                    continue
                if chartName.startswith("blues"):
                    continue
                if chartName.startswith("kids"):
                    continue
                if country in ["USA", "US"]:
                    if chartName in nonUS:
                        continue

                artist = dResults["Artist"]

                renamedArtist = artist
                for testArtist in artistRenames.keys():
                    if artist.find(testArtist) != -1:
                        tmp = renamedArtist
                        renamedArtist = renamedArtist.replace(testArtist, artistRenames.get(testArtist))
                        #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist   

                if artist in ["Original Broadway Cast", "Soundtrack", "Who", "Music From The Television Event", "Various"]:
                    continue        
                if artist in ["Original Broadway Cast Recording", "Original Cast"]:
                    continue

                if artist in ["SANDAIME J SOUL BROTHERS from EXILE TRIBE Feat. Afrojack", "Sech, Darell, Nicky Jam, Ozuna & Anuel AA", "Bethel Music, Jonathan David Helser & Melissa Helser", "Delbert McClinton And Self-Made Men + Dana", "Meduza Featuring GOODBOYS", "VASSY & Disco Fries"]:
                    continue
                    #print("")
                    #print(chartName)
                if artist in ["The JT Project Featuring Najee", "Roger Daltrey With Members Of The Who Band And Orchestra/Keith Levenson", "Gerald Causse & Nicolas Giusti", "King & Prince"]:
                    continue
                    #print("")
                    #print(chartName)
                if artist in ["Dominican Sisters Of Mary, Mother Of The Eucharist", "Eric Buchholz/Slovak National Symphony Orchestra/Naigus/Strange/Rudisill/Peacock/Gonzales"]:
                    continue
                    #print("")
                    #print(chartName)
                if artist in ["Kittel & Co.", "BeBe Winans Featuring Tobbi & Tommi Introducing Kiandra", "Jason Campbell Featuring Robin Campbell With Kirsten Campbell Wood, Saydi Driggers And The Phoenix Boys Choir (Stangelberger)"]:
                    continue
                    #print("")
                    #print(chartName)
                if artist in ["Bruckner Orchester Linz Conducted By Dennis Russell Davies", "Dr. Kokastien", "Mormon Tabernacle Choir/Orchestra At Temple Square"]:
                    continue
                    #print("")
                    #print(chartName)
                if artist in ["Various Artists", "Original Cast Recording"]:
                    continue
                    #print("")
                    #print(chartName)

                    
                chartCounter[chartName] += 1

                album  = dResults["Name"]
                
                if chartData.get(artist) is None:
                    chartData[artist] = Counter()
                chartData[artist][album] += 1


                
                
                if fullChartData.get(artist) is None:
                    fullChartData[artist] = {"Songs": {}, "Albums": {}}
                if chartName.endswith("Albums"):
                    key = "Albums"
                else:
                    key = "Songs"
                if fullChartData[artist][key].get(album) is None:
                    fullChartData[artist][key][album] = {}
                if fullChartData[artist][key][album].get(chartName) is None:
                    fullChartData[artist][key][album][chartName] = {}
                fullChartData[artist][key][album][chartName][date] = 0
        print(len(fullChartData))



    artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in fullChartData.items()}        

    if artistRenames is not None:
        print("Renamed {0} artists".format(len(renameStats)))
        print("Most Common Artists:")
        for item in renameStats.most_common(5):
            print(item)

    saveFile(idata=chartCounter, ifile="chartCounter.p", debug=True)
    saveFile(idata=chartData, ifile="chartData.p", debug=True)


    saveFile(idata=artistAlbumData, ifile="currentBillboardArtistAlbumData.p", debug=True)
    saveFile(idata=fullChartData, ifile="currentBillboardFullChartArtistAlbumData.p", debug=True)

    _, _ = clock("Last Run")

In [47]:
findArtistData(minYear=2000, maxYear=2010, country="US")

/Volumes/Piggy/Charts/data/billboard/results/1958.p	0
/Volumes/Piggy/Charts/data/billboard/results/1959.p	0
/Volumes/Piggy/Charts/data/billboard/results/1960.p	0
/Volumes/Piggy/Charts/data/billboard/results/1961.p	0
/Volumes/Piggy/Charts/data/billboard/results/1962.p	0
/Volumes/Piggy/Charts/data/billboard/results/1963.p	0
/Volumes/Piggy/Charts/data/billboard/results/1964.p	0
/Volumes/Piggy/Charts/data/billboard/results/1965.p	0
/Volumes/Piggy/Charts/data/billboard/results/1966.p	0
/Volumes/Piggy/Charts/data/billboard/results/1967.p	0
/Volumes/Piggy/Charts/data/billboard/results/1968.p	0
/Volumes/Piggy/Charts/data/billboard/results/1969.p	0
/Volumes/Piggy/Charts/data/billboard/results/1970.p	0
/Volumes/Piggy/Charts/data/billboard/results/1971.p	0
/Volumes/Piggy/Charts/data/billboard/results/1972.p	0
/Volumes/Piggy/Charts/data/billboard/results/1973.p	0
/Volumes/Piggy/Charts/data/billboard/results/1974.p	0
/Volumes/Piggy/Charts/data/billboard/results/1975.p	0
/Volumes/Piggy/Charts/data/b

# Chart Analysis

In [48]:
chartCounter = getFile("chartCounter.p")
chartCounter.most_common()

[('adult-contemporary', 571),
 ('adult-pop-songs', 571),
 ('alternative-songs', 571),
 ('country-airplay', 571),
 ('country-songs', 571),
 ('hot-100', 571),
 ('hot-mainstream-rock-tracks', 571),
 ('hot-r-and-b-hip-hop-airplay', 571),
 ('pop-songs', 571),
 ('r-b-hip-hop-songs', 571),
 ('radio-songs', 571),
 ('rap-song', 571),
 ('rhythmic-40', 571),
 ('r-b-hip-hop-albums', 548),
 ('catalog-albums', 539),
 ('independent-albums', 519),
 ('country-albums', 516),
 ('billboard-200', 511),
 ('top-album-sales', 511),
 ('rap-albums', 332),
 ('digital-song-sales', 320),
 ('tastemaker-albums', 263),
 ('rock-albums', 245),
 ('digital-albums', 234),
 ('hard-rock-albums', 179),
 ('alternative-albums', 159),
 ('hot-christian-songs', 108),
 ('rock-airplay', 80),
 ('rock-songs', 80),
 ('americana-folk-albums', 55),
 ('country-digital-song-sales', 49),
 ('r-and-b-hip-hop-digital-song-sales', 49),
 ('rock-digital-song-sales', 49),
 ('social-50', 3)]

In [39]:
chartData = getFile("chartData.p")

In [40]:
chartData

{'Savage Garden': Counter({'I Knew I Loved You': 30}),
 'Faith Hill': Counter({'Breathe': 30,
          'The Way You Love Me': 8,
          "There You'll Be": 12,
          'Cry': 16,
          'Fireflies': 4,
          'Mississippi Girl': 4}),
 'Marc Anthony': Counter({'You Sang To Me': 7,
          'Muy Dentro De Mi': 11,
          'Desde Un Principio -- From The Beginning': 22,
          'Libre': 43,
          'Celos': 11,
          'Mended': 1,
          'Viviendo': 4,
          'Ahora Quien': 16,
          'Amar Sin Mentiras': 15,
          'Valio La Pena': 5}),
 'Don Henley': Counter({'Taking You Home': 4}),
 'BBMak': Counter({'Back Here': 7}),
 'Huey Lewis & Gwyneth Paltrow': Counter({"Cruisin'": 1}),
 '*NSYNC': Counter({'This I Promise You': 13,
          'No Strings Attached': 21,
          'Home For Christmas': 1,
          "It's Gonna Be Me": 7,
          'Bye Bye Bye': 15,
          'Celebrity': 4}),
 'Santana Featuring Rob Thomas': Counter({'Smooth': 15}),
 'Vertical Horiz

In [ ]:

slimResults = Counter()
for artist, artistData in results.items():
    for key, keyData in artistData.items():
        for album, albumData in keyData.items():
            slimResults[artist] += sum({k: len(v) for k,v in albumData.items()}.values())

In [ ]:
saveFile(idata = slimResults, ifile="billboardCounter.p", debug=True)
saveFile(idata = results,     ifile="billboardResults.p", debug=True)

# Get Billboard Results

In [5]:
slimResults = getFile(ifile="billboardCounter.p", debug=True)
fullResults = getFile(ifile="billboardResults.p", debug=True)

Loading data from billboardCounter.p
  --> This file is 96.0kB.
Loading billboardCounter.p
Loading data from billboardResults.p
  --> This file is 1.2MB.
Loading billboardResults.p


# Get Discogs

In [6]:
disc = discogs()
discdf = disc.getMasterSlimArtistDiscogsDB()
artistIDToName = discdf["DiscArtist"].to_dict()
from masterdb import getArtistAlbumsDB, discConv
artistAlbumsDB = getArtistAlbumsDB(disc)

artistNameToID = {}
print("Found {0} ID -> Name entries".format(len(artistIDToName)))
for artistID,artistName in artistIDToName.items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToID)))
mulArts  = multiArtist(cutoff=0.9, discdata=artistNameToID, exact=False)

Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Loading data from /Volumes/Music/Discog/db/MasterSlimArtistDB.p
  --> This file is 53.5MB.
Loading /Volumes/Music/Discog/db/MasterSlimArtistDB.p
Current Time is Tue Jan 21, 2020 19:24:37 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 180.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating 

# Check Renames

In [7]:
singleRenames = getFile(ifile="singleRenames.p", debug=True)
multiRenames  = getFile(ifile="multiRenames.p", debug=True)
knownArtists  = getFile(ifile="artistMap.p", debug=True)

Loading data from singleRenames.p
  --> This file is 107.3kB.
Loading singleRenames.p
Loading data from multiRenames.p
  --> This file is 622B.
Loading multiRenames.p
Loading data from artistMap.p
  --> This file is 158.8kB.
Loading artistMap.p


# Near Renames Artists

In [ ]:
from searchUtils import findNearest
cutoff = 0.9
artistsToGet = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
#for i, (artist, cnt) in enumerate(sorted(slimResults.items(), key=lambda pair: pair[1], reverse=False)):
    if i <= 4500:
        continue
    if i % 250 == 0:
        print("==>",i,len(slimResults))
    if multiRenames.get(artist) is not None:
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if knownArtists.get(artist) is not None:
        continue
    if manualRenames.get(artist) is not None:
        continue

    matches = mulArts.getArtistNames(artist)
    if len(matches) > 1:
        continue

    mdata = getMusicData("DiscArtist", artist)
    if mdata is None:
        results = findNearest(artist, artistNameToID.keys(), num=1, cutoff=cutoff)      
        if len(results) > 0:
            artistsToGet[artist] = results
            print("manualRenames[\"{0}\"] = \"{1}\"".format(artist, results[0]))

In [ ]:
findNearest

In [ ]:
for k,v in artistsToGet.items():
    print("keep[\"{0}\"] = {1}  #{2}".format(k, v[0], v[1]))



# Get Multi Results

In [ ]:
from collections import Counter
cutoff = 0.8
multiMatchResult = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
    if i > 0 and i % 1000 == 0:
        print("Passed",i,'/',len(slimResults))
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    if knownArtists.get(prevArtist) is not None:
        continue
    if manualRenames.get(prevArtist) is not None:
        prevArtist = manualRenames[prevArtist]
    mdata = getMusicData("DiscArtist", prevArtist)
    if isinstance(mdata, DataFrame):
        if mdata.shape[0] <= 1:
            continue
        matches = mdata["Name"].index
        artistAlbums = set(list(fullResults[prevArtist]["Songs"].keys()) + list(fullResults[prevArtist]["Albums"].keys()))
        results = getBestArtistIDMatch(artist, artistAlbums, matches, N=3, cutoff=cutoff)
        if results[2] is not None:
            if results[2] >= cutoff:
                print(i,"/",len(slimResults),'  \t',artist,results)
                multiMatchResult[artist] = list(results[:-1])

In [ ]:
len(multiMatchResult)

In [ ]:
print(len(knownArtists))
knownArtists.update(multiMatchResult)
print(len(knownArtists))
saveFile(idata=knownArtists, ifile="artistMap.p", debug=True)

# Single Results

In [ ]:
knownArtists["The Kingston Trio"]

In [16]:
cutoff = 0.60
singleResult = {}
#for i, (artist, cnt) in enumerate(slimResults.most_common()):
for i, (artist, cnt) in enumerate(sorted(slimResults.items(), key=lambda pair: pair[1], reverse=False)):
    if i > 0 and i % 100 == 0 or i == 100:
        print("Passed",i,'/',len(slimResults))
    if cnt < 1:
        continue
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]        
    if prevArtist is None:
        prevArtist = artist
    if manualRenames.get(prevArtist) is not None:
        prevArtist = manualRenames[prevArtist]
    if knownArtists.get(prevArtist) is not None or knownArtists.get(artist) is not None:
        continue
    artistAlbums = set(list(fullResults[artist]["Songs"].keys()) + list(fullResults[artist]["Albums"].keys()))
    match = mulArts.getArtistNames(artist                                                     )
    if len(match) == 1:
        mdata = getMusicData("DiscArtist", list(match.keys())[0])
        if not isinstance(mdata, DataFrame):
            results = getBestArtistMatch(artist, artistAlbums, N=3, cutoff=cutoff)
            if results[2] is not None:
                if results[2] >= cutoff:
                    print(i,"/",len(slimResults),'  \t',artist,'\t',results)
                    singleResult[artist] = results
                    if len(singleResult) > 1000:
                        break

Passed 100 / 6870
Passed 200 / 6870
Passed 300 / 6870
Passed 400 / 6870
Passed 500 / 6870
Passed 600 / 6870
Passed 700 / 6870
Passed 800 / 6870
Passed 900 / 6870
Passed 1000 / 6870
Passed 1100 / 6870
1121 / 6870   	 24 Hrs 	 ('5245200', '24hrs', 0.62)
Passed 1200 / 6870
Passed 1300 / 6870
Passed 1400 / 6870
Passed 1500 / 6870
Passed 1600 / 6870
Passed 1700 / 6870
Passed 1800 / 6870
1841 / 6870   	 Bryan Bautista 	 ('3388076', 'Juan Bautista', 0.62)
Passed 1900 / 6870
Passed 2000 / 6870
Passed 2100 / 6870
Passed 2200 / 6870
Passed 2300 / 6870
Passed 2400 / 6870
Passed 2500 / 6870
Passed 2600 / 6870
Passed 2700 / 6870
Passed 2800 / 6870
Passed 2900 / 6870
Passed 3000 / 6870
3011 / 6870   	 VASSY 	 ('40430', 'VAST', 0.64)
Passed 3100 / 6870
Passed 3200 / 6870
Passed 3300 / 6870
Passed 3400 / 6870
Passed 3500 / 6870
Passed 3600 / 6870
Passed 3700 / 6870
Passed 3800 / 6870
Passed 3900 / 6870
3943 / 6870   	 Phil Collins and Marilyn Martin 	 ('253972', 'Marilyn Martin', 0.94)
Passed 4000 / 6

In [ ]:
del singleResult["Murk vs. Kristine W"]
del singleResult["Chris Cox Vs. Happy Clappers"]
del singleResult["T.M.Revolution × Nana Mizuki"]
del singleResult["Bob Marley Vs. Funkstar De Luxe"]

In [20]:
singleResults = {k: v[0] for k,v in singleResult.items()}
print("Found {0} single results".format(len(singleResults)))
print("Found {0} all results".format(len(knownArtists)))
knownArtists.update(singleResults)
print("Found {0} all results".format(len(knownArtists)))
saveFile(idata=knownArtists, ifile="artistMap.p", debug=True)

Found 5 single results
Found 6041 all results
Found 6046 all results
Saving data to artistMap.p
  --> This file is 158.9kB.
Saved data to artistMap.p
  --> This file is 158.9kB.


## Known Results

In [ ]:
from collections import Counter
cutoff = 0.8
multiMatchResult = {}
for i, (artist, cnt) in enumerate(slimResults.most_common()):
    if i > 0 and i % 1000 == 0:
        print("Passed",i,'/',len(slimResults))
    prevArtist = None
    if multiRenames.get(artist) is not None:
        prevArtist = artist
        artist = multiRenames[artist]
    if singleRenames.get(artist) is not None:
        idx    = str(singleRenames[artist])
        artist = artistIDToName[idx]
    if prevArtist is None:
        prevArtist = artist
    if knownArtists.get(prevArtist) is not None:
        continue
    mdata = getMusicData("DiscArtist", prevArtist)
    if isinstance(mdata, DataFrame):
        if mdata.shape[0] <= 1:
            continue
        matches = mdata["Name"].index
        artistAlbums = set(list(fullResults[prevArtist]["Songs"].keys()) + list(fullResults[prevArtist]["Albums"].keys()))
        results = getBestArtistIDMatch(artist, artistAlbums, matches, N=3, cutoff=cutoff)
        if results[2] is not None:
            if results[2] >= cutoff:
                print(i,"/",len(slimResults),'  \t',artist,results)
                multiMatchResult[artist] = list(results[:-1])

In [ ]:
singleResults = {k: v[0] for k,v in multiMatchResult.items()}
print("Found {0} single results".format(len(singleResults)))
print("Found {0} all results".format(len(knownArtists)))
knownArtists.update(singleResults)
print("Found {0} all results".format(len(knownArtists)))

In [ ]:
saveFile(idata=knownArtists, ifile="singleRenames.p")

# Check For Misnames

In [ ]:
ma = multiartist(cutoff=0.9, discdata=discdata)

In [ ]:
print(ma.getArtistNames('Lipps, Inc.') == {'Lipps, Inc.': ['159617', '26641', '101850']})
print(ma.getArtistNames('Bob Wills & His Texas Playboys') == {'Bob Wills & His Texas Playboys': ['670000','786114','1134146','804668','804679','1004309','935875','935907']})
print(ma.getArtistNames('Brad Paisley Duet With Carrie Underwood', debug=False) == {'Carrie Underwood': ['1011680'], 'Brad Paisley': ['313755']})
print(ma.getArtistNames('Ray Charles With Willie Nelson', debug=False) == {'Willie Nelson': ['249449'], 'Ray Charles': ['521963', '30552']})

# Album Data

In [ ]:
savename = join(savedir, "billboard.p")
albumData = getFile(join(savedir, "billboard-album.p"))
#chartdata = getFile(savename, debug=True)

In [ ]:
for artist,artistData in albumData.items():
    print(artist)
    for album in artistData.keys():
        print("\t",album)

# Show Data

In [ ]:
togets = {}
combos = []

savedir  = join(basedir, "results")
savename = join(savedir, "billboard.p")
chartdata = getFile(savename, debug=True)
chartArtists = sorted(chartdata.keys(), reverse=True)

import json
prevs = json.load(open("prevs.json", "r"))
artPrev = json.load(open("../discogs/prevs.json", "r"))
prevs.update(artPrev)
json.dump(prevs, open("prevs.json", "w"))
print("There are {0} artists previously searched.".format(len(prevs)))

ma = multiartist(cutoff=0.9, discdata=discdata, exact=False)

from random import shuffle
print("There are {0} chart artists".format(len(chartArtists)))
shuffle(chartArtists)
for i,artist in enumerate(chartArtists):
    if prevs.get(artist) is not None:
        continue
    retval = ma.getArtistNames(artist)
    if retval is None:
        togets[artist] = artist
    else:
        for name,value in retval.items():
            if prevs.get(name) is not None:
                continue
            if value is None:
                togets[name.upper()] = artist
                print("{0: <3}{1: <40}{2: <60}{3}".format(len(togets),name,artist,retval))
                print('\t---->',i,'/',len(chartArtists))

    
    if len(togets) > 50:
        print("\n\n\n\n")
        for name,artist in togets.items():
            print("art.searchDiscogForArtist(\"{0}\")  ## {1}".format(name, artist))
        break

###### artists = [x.strip() for x in vals.split("\n")]
artists = [x for x in artists if len(x) > 0]

In [ ]:
Pitbull Featuring Ne-Yo, Afrojack & Nayer
Drake Featuring Kanye West, Lil Wayne & Eminem

In [ ]:
def testName(name, discdata):
    if discdata.get(name) is not None:
        return discdata[name]
    return name

def findName(name, discArtists, threshold):
    retval = findNearest(name, discArtists, 1, threshold)
    if len(retval) == 0:
        return name
    return retval
    

def splitArtist(name):
    names = [name]
    names = [[x.strip() for x in y.split(" Featuring ")]
results = {}
    
for name in artists:
    if " & " in name:
        retvals = [testName(x, discdata) for x in 
        retvals = [findName(x, discArtists, 0.95) if not isinstance(x, list) else x for x in retvals]
        results[name] = retvals        

In [ ]:
togets = []
for k,v in results.items():
    for val in v:
        if not isinstance(val, list):
            togets.append("art.searchDiscogForArtist(\"{0}\")".format(val))
            
for toget in set(togets):
    print(toget)

In [ ]:
tmp = [['5070865', '90037', '4135543', '1507065'], ['Katy XXX Perry', ['1201210']]]

In [ ]:
tmp

In [ ]:
nameids = {}
namerefs = {}
for name,nameid in iddata.items():
    artist = name
    if name.endswith(")"):
        artist = None
        for x in [-3,-4,-5]:
            if artist is not None:
                continue
            if abs(x) > len(name):
                continue
            if name[x] == "(":
                try:
                    val = int(name[(x+1):-1])
                    artist = name[:x].strip()
                except:
                    continue
          
        if artist is None:
            artist = name

    ref = refdata[name]
            
    if nameids.get(artist) is None:
        nameids[artist] = {}
    nameids[artist][nameid] = 1
    
    if namerefs.get(artist) is None:
        namerefs[artist] = {}
    namerefs[artist][ref] = 1
    

nameids = {k: list(v.keys()) for k,v in nameids.items()}
namerefs = {k: list(v.keys()) for k,v in namerefs.items()}